# Advanced data processing with NumPy

This week we will continue developing our skills using NumPy to process real data.
In the lesson this week we are using daily weather observations from the [Detroit Willow Run Airport](https://en.wikipedia.org/wiki/Willow_Run_Airport) station, near Ypsilanti, Michigan in the U.S.
This data file is in the same format as the one you'll work with in this week's exercise.

## Getting started

### Previewing the data file.

To start, let's have a look at the top few lines of our data file.

So, we have 9 columns in the file, with values separated by variable numbers of spaces.
Another thing to notice are the `-9999` values, which indicate missing data.
Otherwise, some columns contain numerical values, and those will be our focus for today's lesson.
If you prefer, you're welcome to open the data file in the JupyterLab editor to explore more.

### Importing NumPy and loading the data

Now let's import NumPy and read in our data to a variable called `data`.

In [1]:
import numpy as np

Now we can read the data file.
Note that we only want the `DATE`, `TAVG`, `TMAX`, and `TMIN` values, which we can select using the `usecols()` parameter.
Don't forget about skipping the header!

In [2]:
fp = '1495566.txt'
data = np.genfromtxt(fp, skip_header=2, usecols=(4, 6, 7, 8))

Now let's see what we're dealing with...

In [3]:
print(data)

[[ 2.0070101e+07 -9.9990000e+03  5.0000000e+01  3.2000000e+01]
 [ 2.0070102e+07 -9.9990000e+03  4.3000000e+01  2.6000000e+01]
 [ 2.0070103e+07 -9.9990000e+03  4.6000000e+01  3.0000000e+01]
 ...
 [ 2.0171229e+07 -9.9990000e+03  2.0000000e+01  1.1000000e+01]
 [ 2.0171230e+07 -9.9990000e+03  1.7000000e+01  4.0000000e+00]
 [ 2.0171231e+07 -9.9990000e+03  1.7000000e+01 -3.0000000e+00]]


### Converting the missing data to nan values

Our first processing task will be to convert the missing data values to `np.nan` values, which will make it easier to use masks later in our data processing.
We can go ahead and do this to all of the data to start, following a similar approach to how we used masks in the last lesson.

First, we need to identify a useful test for identifying missing data.
We could could convert the data to integer values everywhere and test for values being equal to `-9999`, but there's an even easier option.
Since we know all of our data are dates or temperatures, we can simply look for numbers below `-9998` to identify missing data.

In [4]:
data_mask = (data < -9998)
data[data_mask] = np.nan

In [5]:
print(data)

[[ 2.0070101e+07            nan  5.0000000e+01  3.2000000e+01]
 [ 2.0070102e+07            nan  4.3000000e+01  2.6000000e+01]
 [ 2.0070103e+07            nan  4.6000000e+01  3.0000000e+01]
 ...
 [ 2.0171229e+07            nan  2.0000000e+01  1.1000000e+01]
 [ 2.0171230e+07            nan  1.7000000e+01  4.0000000e+00]
 [ 2.0171231e+07            nan  1.7000000e+01 -3.0000000e+00]]


Now we see the expected `nan` values in place of the `-9999` values.
Note that a `nan` value in NumPy is refered to as `np.nan`.

### Splitting the data into column arrays

Like the last lesson, we can now separate our data into different arrays with meaningful names.
For example, we can create a `date` array using the first column of `data`.
Remember that we'll do this using array slicing.

In [6]:
date = data[:,0]

In [7]:
print(date)

[20070101. 20070102. 20070103. ... 20171229. 20171230. 20171231.]


Looks good, let's do the others.

In [8]:
tavg = data[:,1]
tmax = data[:,2]
tmin = data[:,3]

## Examining our data

Now that we have loaded the data, let's have a look at what we're dealing with.

### Basic observations

We can start by looking at some basic information, such as the starting and ending dates for the observations.

In [9]:
print(date.min())

20070101.0


In [10]:
print(date.max())

20171231.0


OK, so we can see our observations start on January 1, 2007 and continue until December 31, 2017.

### Checking for missing/bad data

Let's make sure that we don't have any missing dates by looking for any `nan` values in the `date` array.
To do this, we'll need a new NumPy function we've not yet used called `np.count_nonzero()`.
`np.count_nonzero()` does exactly what is says, counts all values that are not equal to zero in an array and returns the sum.
In this case, `False` values in mask arrays, for example, are considered to be equal to zero in the count.
Thus, if we have a mask array with all of the *finite* values for an array like `date`, that should give `np.count_nonzero()` will give us the number of dates with useful data.
Let's see how it works.

In [11]:
date_mask = np.isfinite(date)

In [12]:
print("Number of days:", np.count_nonzero(date_mask))

Number of days: 4016


OK, so we have 4016 dates in the data we're using, but how do we know whether we're missing any dates?
One way to check is to take the opposite of the mask array values by using the `~` character, the logical not operator.
How does it work?
Let's see.

In [13]:
missing_date_mask = ~np.isfinite(date)

In [14]:
print("Number of missing days:", np.count_nonzero(missing_date_mask))

Number of missing days: 0


OK, so no missing days, which is not really a big surprise.
Good to know at least.
How about for the other arrays like `tavg`?

In [15]:
tavg_mask = np.isfinite(tavg)
print("Number of average temps:", np.count_nonzero(tavg_mask))

Number of average temps: 0


Oh no!
So we don't have any average temperature observations.
Looks like we'll have to use another array for exploring the temperature data.

### Calculations within a date range

One common thing we'd like to do is calculate something about our data over a fixed time range, for example, one year.
We've seen this idea previously, but just to remind you we can do this by masking our data.
First, let's check for any missing data in the `tmax` array.

In [16]:
missing_tmax_mask = ~np.isfinite(tmax)
print("Number of missing tmax values:", np.count_nonzero(missing_tmax_mask))

Number of missing tmax values: 7


OK, so we need to remove the missing values first, which we can do as we've done before: Make a mask array, then remove bad values using the mask.

In [17]:
tmax_mask = np.isfinite(tmax)

In [18]:
tmax_clean = tmax[tmax_mask]

We can also remove the corresponding dates by using the same mask (`tmax_mask`) on our `date` array.

In [19]:
date_clean = date[tmax_mask]

### Average annual temperature, method 1

OK, now let's use a range of dates to find the average maximum temperature for one year.
In this case, let's go for 2010.

In [20]:
tmax_2010 = tmax[(date >= 20100101) & (date <= 20101231)]

In [21]:
tmax_2010.mean()

60.8027397260274

OK, so this works, but we'll see another more flexible approach in just a moment.

### Average monthly temperatures

So one possibility is for us to use numerical ranges to calculate things like average annual temperatures.
An alternative is to split date into different arrays, with one for each year, month, and day in the data.
To do this, we first need to convert the `date_clean` array to be character string values rather than `float64` values.
Let's do that.

In [22]:
date_clean.astype(str)

array(['20070101.0', '20070102.0', '20070103.0', ..., '20171229.0',
       '20171230.0', '20171231.0'], dtype='<U32')

Well, that's not quite what we want.
Ideally, we want a date in the form `YYYYMMDD` where `YYYY` is the year, `MM` is the month, and `DD` is the day.
Looks like we need to convert the date to an integer first.

In [23]:
date_clean_int = date_clean.astype(int)

In [24]:
date_clean_str = date_clean_int.astype(str)

In [25]:
print(date_clean_str)

['20070101' '20070102' '20070103' ... '20171229' '20171230' '20171231']


That's better.

### Splitting dates into separate year, month, day

Now we can go about splitting our dates into separate arrays for the year, month, and day.
As a reminder, it is first useful to recall that character strings can be sliced using index values just like arrays.

In [26]:
date_fmt = "YYYYMMDD"

In [27]:
date_fmt[0:4]

'YYYY'

So by taking the first 4 values from the string `YYYYMMDD` we get `YYYY`.
We can do the same for the month, for example.

In [28]:
date_fmt[4:6]

'MM'

With this in mind, we can now attempt the same in our NumPy array `date_clean_str`.

In [29]:
year = [datenow[0:4] for datenow in date_clean_str]

In [30]:
year = np.array(year)

In [31]:
print(year)

['2007' '2007' '2007' ... '2017' '2017' '2017']


OK, so we have the year now, but what happened with the `[datenow[0:4] for datenow in date_clean_str]` stuff.
We've never seen that before.
You're right, this is new, and is an example of list comprehension in Python.
Basically, it is a shorthand approach for looping over all of the values and doing something to them.
In our case, we find the first 4 characters for each value in the `date` array.

To be more clear,
```python
[datenow[0:4] for datenow in date_clean_str]
```
is equivalent to doing
```python
year = []
for datenow in date_clean_str:
    year.append(datenow[0:4]
```
since this would produce a new list called `year`, we then need to use the `np.array()` function to convert that list to a NumPy array.
Let's do the same for the months and days.

In [32]:
month = [datenow[4:6] for datenow in date_clean_str]
month = np.array(month)

In [33]:
day = [datenow[6:8] for datenow in date_clean_str]
day = np.array(day)

### Finding the average monthly max temperature

Now that we have separate arrays for the year, month, and day we can do some fun stuff like calculate the monthly average temperature for a given year.
Let's take 2010 again as our example and find the average temperatures for each month in 2010.
For this, we can use a `for` loop.

In [34]:
means_2010 = np.zeros(12)
index = 0

In [35]:
for month_now in np.unique(month):
    means_2010[index] = tmax_clean[month == month_now].mean()
    index = index + 1

In [36]:
print(means_2010)

[32.71847507 34.0483871  47.72941176 61.29393939 73.50439883 82.15805471
 86.02058824 84.08554572 77.43030303 64.65982405 50.8662614  37.66961652]


OK, so let's break this down as there were a few things that happened here that might be unfamiliar.

1. We created an array of zeros with 12 locations to store the monthly average temperatures.
2. We used an index variable called `index` to define where to store those monthly average temperatures. We could have done this differently, but there is a good reason to use `index` here because it is more flexible if we wanted to consider more than one year of data as you'll see in this week's exercise.
3. We then used a `for` loop to go through each unique month (found using `np.unique()`).
4. For each month we found the max temperature using the condition `month == month_now`, and taking the `tmax_clean.mean()` for that month.
5. We added `1` to the `index` value to ensure the next mean value goes into the correct location in the `means_2010` array.

This might seem a bit complicated, but it is rather powerful as well.
We've efficiently calcuated mean monthly max temperatures in only a few lines of code!

### Using functions on NumPy array data

As you might have noticed above, the `means_2010` array has temperatures in Fahrenheit, which might be easier for us to work with in Celsius.
Fortunately, we can easily convert NumPy array data from Fahrenheit to Celsius using a familiar function.

In [37]:
def fahrToCelsius(temp):
    return (temp - 32.0) / 1.8

Now, let's go about that conversion.

In [38]:
means_2010_celsius = fahrToCelsius(means_2010)

In [39]:
print(means_2010_celsius)

[ 0.39915282  1.13799283  8.73856209 16.27441077 23.05799935 27.86558595
 30.01143791 28.93641429 25.23905724 18.14434669 10.48125633  3.14978696]


Finally, something easy to do with NumPy!
All we have to do hear is plug in a NumPy array with our function and we get back the converted temperature values.

### Computing monthly average temperatures for a range of years

One thing we'll encounter in this week's exercise is a need to calculate monthly average temperatures for a number of years.
For example, the average temperature in February from 2010-2015.
We can do that with the tools we have, but in a slightly different way than calculating the monthly average temperature for each month in those years one by one.
First, let's make a few masks, starting with the month.

In [40]:
month_mask = month == '02'

Nothing crazy here, just ensuring the month is February, or `'02'`.
Now for the years.

In [41]:
year_mask = (year.astype(int) >= 2010) & (year.astype(int) <= 2015)

Again, nothing too scary here.
We do need to use the `.astype(int)` here to be able to check the range of years, but that is not so bad.
Now let's find that monthly average temperature for February between 2010 and 2015.

In [42]:
mean_2010_2015 = tmax_clean[month_mask & year_mask].mean()

In [43]:
print(mean_2010_2015)

32.38690476190476


Looks about right, but perhaps we can check the value in Celsius to see whether that makes sense for winter in the northern hemisphere.

In [44]:
print(fahrToCelsius(mean_2010_2015))

0.21494708994708844


Right around 0°C, looks good!

### Comparing monthly temperatures

One thing we can do now that we have a mean temperature for February in 2010-2015 is to compare one of those years to the average max temperature in the time period.
This would give us a temperature anomaly, the difference between the mean value and a given year.
Let's give that a shot.

In [45]:
anomaly = means_2010[1] - mean_2010_2015

In [46]:
print(anomaly)

1.661482334869433


As we can see, February of 2010 was 1.7° Fahrenheit warmer than the average February between 2010 and 2015.
We'll explore exactly this kind of thing in more detail in this week's exercise.